In [1]:
import os
import snowflake.connector as sf
import pandas as pd
import access
import sqlite3

# O caminho do diretório onde você deseja criar os arquivos SQL
output_directory = "C:/ProjetosPython/EDD-DL/Gerar_Script_Sql/arquivos/procs_carga_curated/"

# Certifique-se de que o diretório exista, senão o crie
os.makedirs(output_directory, exist_ok=True)

# Conexão SQLite
conn = sqlite3.connect('C:/ProjetosPython/EDD-DL/Gerar_Script_Sql/database/config_scripts.db')
cursor = conn.cursor()

# Leitura das tabelas ativas do SQLite
query_tabelas = "SELECT * FROM config_tabelas WHERE ATIVO = 'true';"
df_tabelas = pd.read_sql_query(query_tabelas, conn)
#print(df_tabelas_full)


for tabelas in df_tabelas['TABELA']:

    query_estrutura_sf_t= f"""
        SELECT COLUMN_NAME
        FROM INFORMATION_SCHEMA.COLUMNS
        WHERE TABLE_CATALOG = 'RESSARCIMENTO'
        AND TABLE_SCHEMA = 'TRANSIENT'     
        AND COLUMN_NAME NOT IN ('ID_ORIGINAL', 'ID_DATABASE')
        AND TABLE_NAME = 'STG_{tabelas.upper()}'
        ORDER BY ORDINAL_POSITION;
    """

    query_estrutura_sf_c= f"""
        SELECT COLUMN_NAME
        FROM INFORMATION_SCHEMA.COLUMNS
        WHERE TABLE_CATALOG = 'RESSARCIMENTO'
        AND TABLE_SCHEMA = 'CURATED_DATA'        
        AND COLUMN_NAME NOT IN ('ID', 'ID_DATABASE')
        AND TABLE_NAME = '{tabelas.upper()}'
        ORDER BY ORDINAL_POSITION;
    """
    #print(query_estrutura_sf_c)
    #print(query_estrutura_sf_t)
    try:
        # Obtém a estrutura das colunas CURATED e STAGE no Snowflake
        with sf.connect(
            user=access.USER_SF,
            password=access.PWD_SF,
            account=access.ACCOUNT_SF
        ) as sfConnection:
            sfq = sfConnection.cursor()
            # Adicione o comando USE DATABASE aqui
            sfq.execute("USE DATABASE RESSARCIMENTO")
           
             # Obtém a estrutura das colunas CURATED no Snowflake  
            sfq.execute(query_estrutura_sf_c)
            colunas_sf_c = sfq.fetchall()
            #print(colunas_sf_c)

            # Obtém a estrutura das colunas STAGE no Snowflake  
            sfq.execute(query_estrutura_sf_t)
            colunas_sf_t = sfq.fetchall()
            #print(colunas_sf_t)

    except Exception as err:
        print(err)      

    # Transforme as colunas obtidas em uma lista de nomes
    colunas_curated = ', '.join([col[0] for col in colunas_sf_c])
    colunas_stage = ', '.join([f"STG_{tabelas}.{col[0]}" for col in colunas_sf_t])
    
      # Criação do conteúdo do arquivo SQL
    sql_content = f"""CREATE OR REPLACE PROCEDURE RESSARCIMENTO.TRANSIENT.P_STG_{tabelas}()
                        RETURNS VARCHAR(16777216)
                        LANGUAGE SQL
                        EXECUTE AS OWNER
                        AS '

                        BEGIN

                        --CRIA A TEMP COM OS IDS DA STAGE
                            execute immediate ''CREATE OR REPLACE TEMPORARY TABLE RESSARCIMENTO.TRANSIENT.T_TMP_STG_{tabelas} AS
                            SELECT A.ID AS ID_ORIGINAL,
                            A.ID_DATABASE
                            FROM RESSARCIMENTO.TRANSIENT.STG_{tabelas} A
                            LEFT JOIN RESSARCIMENTO.CURATED_DATA.{tabelas} B
                            ON A.ID = B.ID_ORIGINAL
                            AND A.ID_DATABASE = B.ID_DATABASE
                            WHERE B.ID_ORIGINAL IS NOT NULL; '';

                        --APAGA TUDO DA CURATED
                            execute immediate ''DELETE
                            FROM RESSARCIMENTO.CURATED_DATA.{tabelas} AS CURATED
                            USING RESSARCIMENTO.TRANSIENT.T_TMP_STG_{tabelas} AS STG
                            WHERE CURATED.ID_ORIGINAL = STG.ID_ORIGINAL
                            AND CURATED.ID_DATABASE = STG.ID_DATABASE; '';

                        --INSERE NA CURATED                  
                            execute immediate ''
                        INSERT INTO RESSARCIMENTO.CURATED_DATA.{tabelas} ({colunas_curated}, ID_DATABASE)
                            SELECT {colunas_stage}, ID_DATABASE FROM RESSARCIMENTO.TRANSIENT.STG_{tabelas};'';
                                                
                        END';
    """
    #print(sql_content)
    
   #print(sql_content)
    output_file_name = tabelas + ".sql"
        # Caminho completo do arquivo
    output_path = os.path.join(output_directory, output_file_name)
    with open(output_path, "a", encoding='utf8') as file:
            file.write(sql_content)
            file.close()
    print("Arquivos SQL gerados com sucesso! " +str(output_file_name))


Arquivos SQL gerados com sucesso! motivos_fat.sql
Arquivos SQL gerados com sucesso! pbm_vendas_pedidos.sql
Arquivos SQL gerados com sucesso! pbm_vendas_repostas.sql
Arquivos SQL gerados com sucesso! produtos_pedidos.sql
Arquivos SQL gerados com sucesso! produtos_notas.sql
Arquivos SQL gerados com sucesso! clientes_area.sql
Arquivos SQL gerados com sucesso! sortimentos.sql
Arquivos SQL gerados com sucesso! tipos_cliente.sql
Arquivos SQL gerados com sucesso! sub_produtos_pedidos.sql
Arquivos SQL gerados com sucesso! divisao_produto.sql
Arquivos SQL gerados com sucesso! condicoes_fat.sql
Arquivos SQL gerados com sucesso! grupos_clientes.sql
Arquivos SQL gerados com sucesso! area.sql
Arquivos SQL gerados com sucesso! familia.sql
Arquivos SQL gerados com sucesso! notas_sub_pedidos.sql
Arquivos SQL gerados com sucesso! sub_pedidos.sql
Arquivos SQL gerados com sucesso! pedido_direto_produtos.sql
Arquivos SQL gerados com sucesso! op_logisticos.sql
Arquivos SQL gerados com sucesso! pedidos.sql
